# 实验说明

本实验基于`naturalcc`工具集，它是业界首个专注代码智能的深度模型开源训练工具集，包含了对业界现有的代码模型和下游任务的性能评价。相关论文发表在软件工程顶级会议ICSE上，参考文献 [NaturalCC: An Open-Source Toolkit for Code Intelligence](https://xcodemind.github.io/papers/icse22_naturalcc_camera_submitted.pdf)，实验要求使用`naturalcc`工具集复现类型推导任务中的经典方法`Typilus`，此方法发表在2020年的程序语言顶级会议PLDI上，相关文献为[Typilus: neural type hints](https://arxiv.org/pdf/2004.10657.pdf)。

实验分为五个部分，环境准备、数据获取、模型和超参数配置、模型训练与评价、拓展研究。

完整复现Typilus需要计算节点的内存大于或等于`128GB`，显存大于或等于 `32GB`，如无计算节点，建议裁剪数据集，以及使用 [Google Colab](http://colab.research.google.com) (可以提供12GB的内存和一张16GB显存的 Tesla T4显卡）

为避免包冲突，建议使用[Anaconda](http://anaconda.org)管理Python环境。

# 环境准备

## NaturalCC 环境

首先在GitHub上下载`naturalcc`工具集，执行下面的命令

In [ ]:
!git clone https://github.com/CGCL-codes/naturalcc.git

之后在本地安装naturalcc环境，需要提前安装pytorch，此命令只需要执行一次。
推荐使用 Anaconda 虚拟环境容器，避免影响本地Python环境。Anaconda可以在[这里](https://www.anaconda.com/)获取。

In [ ]:
%conda install pytorch torchvision torchaudio cpuonly -c pytorch

In [ ]:
%conda install -c dglteam dgl

Python要求最低版本3.8，安装完成后，执行下面的命令

In [ ]:
%cd naturalcc
%pip install -r requirements.txt
%pip install --editable .
%cd ..

### [GPU训练] 需要注意的地方

训练过程依赖[pytorch库](https://pytorch.org)和[dgl库](https://www.dgl.ai)，如果使用GPU+CUDA训练，需要确保所安装的库版本与本机的CUDA版本对应，例如CUDA 11.0+需要安装```torch+cu110, dgl-cu110```，如果仅使用CPU训练，则只需要安装CPU版本的`torch`和`dgl`即可

在使用GPU训练前需要通过`gpustat`或`nvidia-smi`检查GPU是否已经正确安装，如下所示

In [ ]:
!gpustat

## Typilus环境

构建Typilus数据集需要使用`docker`，在电脑上安装`docker`后，获取`typilus`的源代码

In [ ]:
!git clone https://github.com/typilus/typilus.git

之后在 `typilus/src/data_preparation`目录构建docker镜像

In [ ]:
%cd typilus/src/data_preparation/
# 先需要下载docker
!docker build -t typilus-env .

# 数据获取

## Typilus Graph 构建

在开始训练之前，我们首先需要获得训练所需的数据，我们将首先使用`Typilus`内置的数据处理程序将Python数据转换为Typilus Graph，然后把graph导入`naturalcc`进行训练和预测

运行刚才构建的docker镜像，并设置Typilus训练数据保存的位置

In [ ]:
# windows下要加 ""
!docker run --rm -it -v "$(pwd)/data:/usr/data" typilus-env:latest bash

在Docker shell中，输入下面的命令来构建数据集

``bash scripts/prepare_data.sh metadata/typedRepos.txt, 这一步在windows上会出现问题``

*注意：这条命令可能会执行若干天，关于命令执行时间过长和死循环的问题，可以参考[这里](https://github.com/typilus/typilus/issues/1)*

获取的数据保存在 `xxx/yyy/zzz/graph-dataset-split` 中，使用Tree命令观察

In [ ]:
!tree /mnt/gold/bizq/Typilus_data/graph-dataset-split

## NaturalCC 数据处理

接下来把生成的Typilus Graph导入NaturalCC

数据的处理过程包含了两个阶段，首先将原始数据整理为naturalcc统一的格式，接下来对数据进行binarize，以适合模型训练。

In [ ]:
import ncc_dataset
ncc_dataset.prepare_dataset('typilus', typilus_path="/mnt/gold/bizq/Typilus_data")

In [ ]:
ncc_dataset.binarize_dataset('typilus')

# 模型训练

我们已经提供了训练代码，模型训练的过程只需要执行代码即可。

In [ ]:
!python naturalcc/run/type_prediction/typilus/train.py

In [ ]:
!gpustat

# 模型评价

**挑战1**. 我们在训练代码中插入了模型的评价代码，试着用它们评价模型的准确率！

# 拓展研究

**挑战2**. 调整Typilus模型的超参数（在`config/typilus.yml`中），试着提高模型的训练准确率。

**挑战3**. 利用`naturalcc`中的其他代码，比较在同样的数据集上，`lstm`,`transformer`,`typilus` 的模型预测能力。

**挑战4**. 修改naturalcc的代码，支持[LAMBDANET](https://arxiv.org/pdf/2005.02161.pdf), [Type4Py](https://arxiv.org/pdf/2101.04470.pdf), [Plato](https://arxiv.org/pdf/2107.00157.pdf) 等模型。